# Laboratorio 6 
María Inés Vásquez Figueroa 18250\
Paula Camila González Ortega 18398

In [2]:
import json
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [3]:
# Se abre el archivo JSON
f = open('large_eve.json')
info = f.readlines()

In [4]:
print('Cantidad de registros: ', len(info))
# Se cierra el archivo
f.close()

Cantidad de registros:  746909


In [5]:
# se cargan los datos en el json
jsons_array = [json.loads(line) for line in info if line]
print(jsons_array[0:2])

[{'timestamp': '2017-07-22T17:33:15.784100-0500', 'flow_id': 283570730437312, 'pcap_cnt': 18519, 'event_type': 'alert', 'vlan': 130, 'src_ip': '192.168.203.200', 'src_port': 2328, 'dest_ip': '192.181.145.109', 'dest_port': 445, 'proto': 'TCP', 'alert': {'action': 'allowed', 'gid': 1, 'signature_id': 2001569, 'rev': 15, 'signature': 'ET SCAN Behavioral Unusual Port 445 traffic Potential Scan or Infection', 'category': 'Misc activity', 'severity': 3}}, {'timestamp': '2017-07-22T17:33:16.571273-0500', 'flow_id': 1519048138171115, 'pcap_cnt': 21531, 'event_type': 'ssh', 'vlan': 140, 'src_ip': '192.168.204.60', 'src_port': 62990, 'dest_ip': '192.168.26.254', 'dest_port': 22, 'proto': 'TCP', 'ssh': {'client': {'proto_version': '2.0', 'software_version': 'OpenSSH_5.0'}, 'server': {'proto_version': '1.99', 'software_version': 'Cisco-1.25'}}}]


#### Se filtran los registros DNS

In [6]:
# Funcion para solo el los datos de JSON con cierto caracteristica
def filtro_json(dict_jsons, criterio):
    def iteracion(x):
        if criterio in list(x.keys()):
            # se busca dentro el criterio de keys de los jsons
            return True
        return False
    return filter(iteracion, dict_jsons)

In [7]:
# se filtra los registros dns
data_filtrada = list(filtro_json(jsons_array, "dns"))

In [9]:
print('Cantidad de registros filtrados: ', len(data_filtrada))

Cantidad de registros filtrados:  15749


In [13]:
print('Ejemplo de registros DNS:\n', data_filtrada[4], '\n\n', data_filtrada[9])

Ejemplo de registros DNS:
 {'timestamp': '2017-07-22T17:33:27.380146-0500', 'flow_id': 578544790391795, 'pcap_cnt': 54520, 'event_type': 'dns', 'vlan': 150, 'src_ip': '192.168.207.4', 'src_port': 53, 'dest_ip': '192.168.205.170', 'dest_port': 31393, 'proto': 'UDP', 'dns': {'type': 'answer', 'id': 54724, 'rcode': 'NXDOMAIN', 'rrname': '<root>', 'rrtype': 'SOA', 'ttl': 20864}} 

 {'timestamp': '2017-07-22T17:33:45.672579-0500', 'flow_id': 541745511744323, 'pcap_cnt': 56307, 'event_type': 'dns', 'vlan': 120, 'src_ip': '192.168.202.172', 'src_port': 55580, 'dest_ip': '192.168.207.4', 'dest_port': 53, 'proto': 'UDP', 'dns': {'type': 'query', 'id': 12854, 'rrname': 'fxfeeds.mozilla.com', 'rrtype': 'A', 'tx_id': 0}}


#### Normalizacion y creacion del dataframe

In [8]:
df = pd.json_normalize(data_filtrada)
df.head()

,timestamp,flow_id,pcap_cnt,event_type,vlan,src_ip,src_port,dest_ip,dest_port,proto,dns.type,dns.id,dns.rrname,dns.rrtype,dns.tx_id,dns.rcode,dns.ttl,dns.rdata
0,2017-07-22T17:33:16.661646-0500,1327836194150542,22269,dns,110,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,59680,2001:0500:0001:0000:0000:0000:803f:0235,53,UDP,query,15529,api.wunderground.com,A,0.0,NaN,NaN,NaN
1,2017-07-22T17:33:24.990320-0500,2022925111925872,54352,dns,110,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,38051,2001:0500:0003:0000:0000:0000:0000:0042,53,UDP,query,58278,stork79.dropbox.com,A,0.0,NaN,NaN,NaN
2,2017-07-22T17:33:27.379891-0500,578544790391795,54519,dns,150,192.168.205.170,31393,192.168.207.4,53,UDP,query,54724,hpca-tier2.office.aol.com.ad.aol.aoltw.net,A,0.0,NaN,NaN,NaN
3,2017-07-22T17:33:27.380146-0500,578544790391795,54520,dns,150,192.168.207.4,53,192.168.205.170,31393,UDP,answer,54724,hpca-tier2.office.aol.com.ad.aol.aoltw.net,NaN,NaN,NXDOMAIN,NaN,NaN
4,2017-07-22T17:33:27.380146-0500,578544790391795,54520,dns,150,192.168.207.4,53,192.168.205.170,31393,UDP,answer,54724,<root>,SOA,NaN,NXDOMAIN,20864.0,NaN


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15749 entries, 0 to 15748
Data columns (total 18 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   timestamp   15749 non-null  object 
 1   flow_id     15749 non-null  int64  
 2   pcap_cnt    15749 non-null  int64  
 3   event_type  15749 non-null  object 
 4   vlan        15749 non-null  int64  
 5   src_ip      15749 non-null  object 
 6   src_port    15749 non-null  int64  
 7   dest_ip     15749 non-null  object 
 8   dest_port   15749 non-null  int64  
 9   proto       15749 non-null  object 
 10  dns.type    15749 non-null  object 
 11  dns.id      15749 non-null  int64  
 12  dns.rrname  15749 non-null  object 
 13  dns.rrtype  10921 non-null  object 
 14  dns.tx_id   6056 non-null   float64
 15  dns.rcode   9693 non-null   object 
 16  dns.ttl     4865 non-null   float64
 17  dns.rdata   26 non-null     object 
dtypes: float64(2), int64(6), object(10)
memory usage: 2.2+ MB


#### Buscando dominios DGA

In [10]:
df['dns.rrtype'].value_counts()

SOA     4852
A       2849
AAAA    1978
TXT      789
PTR      427
SRV       13
NS        13
Name: dns.rrtype, dtype: int64

In [11]:
filtro_registrosA = df[df['dns.rrtype'] == 'A'] # los DGA son aquellos registros tipo A
filtro_registrosA['dns.rrtype'].value_counts()

A    2849
Name: dns.rrtype, dtype: int64

#### Se filtran los dominios unicos

In [12]:
dominios_unicos = filtro_registrosA.drop_duplicates(subset=['dns.rrname'])
dominios_unicos.tail()

,timestamp,flow_id,pcap_cnt,event_type,vlan,src_ip,src_port,dest_ip,dest_port,proto,dns.type,dns.id,dns.rrname,dns.rrtype,dns.tx_id,dns.rcode,dns.ttl,dns.rdata
15632,2017-07-22T19:44:29.056834-0500,787856241843714,4405786,dns,130,192.168.203.69,60088,192.168.207.4,53,UDP,query,10109,ntp.ubuntu.com.localdomain,A,0.0,NaN,NaN,NaN
15661,2017-07-22T19:45:10.437871-0500,1612464195546735,4417023,dns,160,192.168.207.4,11162,192.168.206.44,53,UDP,query,46237,redir.metaservices.microsoft.com,A,0.0,NaN,NaN,NaN
15701,2017-07-22T19:47:38.301392-0500,515168780589392,4452953,dns,120,192.168.202.172,51325,192.168.207.4,53,UDP,query,54310,ocsp.verisign.com,A,0.0,NaN,NaN,NaN
15716,2017-07-22T19:39:02.426497-0500,572502265201153,4319593,dns,180,192.168.198.58,42157,192.168.207.4,53,UDP,query,33079,192.168.22.201:.stayonline.net,A,0.0,NaN,NaN,NaN
15737,2017-07-22T19:42:44.728139-0500,1137256119409739,4380897,dns,120,192.168.202.172,50164,192.168.207.4,53,UDP,query,59436,client-software.real.com,A,0.0,NaN,NaN,NaN


In [13]:
doms = dominios_unicos['dns.rrname'].value_counts()
doms

api.wunderground.com                      1
www.cakephp.org                           1
mirror.sanctuaryhost.com                  1
mirror.team-cymru.org                     1
mirror.umoss.org                          1
                                         ..
sourceforge.net                           1
www.freepbx.org                           1
secure.informaction.com.hackerlabs.vpn    1
www.bigflickrfeed.com                     1
client-software.real.com                  1
Name: dns.rrname, Length: 177, dtype: int64

####  TLD (top level domain)

In [14]:
from flare.data_science.features import domain_tld_extract
# Domain TLD Extract
dominios_unicos['tld_extract_domain'] = dominios_unicos['dns.rrname'].apply(domain_tld_extract)

In [15]:
dominios_unicos.head()

,timestamp,flow_id,pcap_cnt,event_type,vlan,src_ip,src_port,dest_ip,dest_port,proto,dns.type,dns.id,dns.rrname,dns.rrtype,dns.tx_id,dns.rcode,dns.ttl,dns.rdata,tld_extract_domain
0,2017-07-22T17:33:16.661646-0500,1327836194150542,22269,dns,110,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,59680,2001:0500:0001:0000:0000:0000:803f:0235,53,UDP,query,15529,api.wunderground.com,A,0.0,NaN,NaN,NaN,wunderground.com
1,2017-07-22T17:33:24.990320-0500,2022925111925872,54352,dns,110,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,38051,2001:0500:0003:0000:0000:0000:0000:0042,53,UDP,query,58278,stork79.dropbox.com,A,0.0,NaN,NaN,NaN,dropbox.com
2,2017-07-22T17:33:27.379891-0500,578544790391795,54519,dns,150,192.168.205.170,31393,192.168.207.4,53,UDP,query,54724,hpca-tier2.office.aol.com.ad.aol.aoltw.net,A,0.0,NaN,NaN,NaN,aoltw.net
6,2017-07-22T17:33:38.537426-0500,2167545251640146,55687,dns,180,192.168.198.62,35092,192.168.207.4,53,UDP,query,7425,safebrowsing.clients.google.com.home,A,0.0,NaN,NaN,NaN,safebrowsing.clients.google.com.home
9,2017-07-22T17:33:45.672579-0500,541745511744323,56307,dns,120,192.168.202.172,55580,192.168.207.4,53,UDP,query,12854,fxfeeds.mozilla.com,A,0.0,NaN,NaN,NaN,mozilla.com


### Parte 2
Uso de clasificador entrenado en lab 1, y clasificacion de los dominios del df. 

In [16]:
from urllib.parse import urlparse, urlencode

# Funciones utiles para el clasificador
def getDomain(url):
    return urlparse(url).netloc
    
def getProtocol(url):
    return 1 if urlparse(url).scheme == 'https' else 0
    
def getSpecialCharacters(url):
    count_characters = 0
    count_characters += url.count('.')
    count_characters += url.count('-')
    count_characters += url.count('@')
    count_characters += url.count('?')
    count_characters += url.count('&')
    count_characters += url.count('|')
    count_characters += url.count('=')
    count_characters += url.count('_')
    count_characters += url.count('~')
    count_characters += url.count('%')
    count_characters += url.count('/')
    count_characters += url.count('*')
    count_characters += url.count(':')
    count_characters += url.count(',')
    count_characters += url.count(';')
    count_characters += url.count('$')
    count_characters += url.count('%20')
    count_characters += url.count(' ')
    return count_characters

In [17]:
# Se crea el nuevo data frame con los datos del DNS
newDataFrame = pd.DataFrame()

newDataFrame['url_len'] = (dominios_unicos['dns.rrname'].str.len()).fillna(0) # Se elimina todos los NaN encontrados y sustituyen por 0-
newDataFrame['url_ratioDG'] = (dominios_unicos['dns.rrname'].str.count('[0-9]') / dominios_unicos['dns.rrname'].str.len()).fillna(0)
newDataFrame['domain_ratioDG'] = (dominios_unicos['dns.rrname'].apply(getDomain).str.count('[0-9]') / dominios_unicos['dns.rrname'].apply(getDomain).str.len()).fillna(0)

In [18]:
newDataFrame

,url_len,url_ratioDG,domain_ratioDG
0,20,0.000000,0.0
1,19,0.105263,0.0
2,42,0.023810,0.0
6,36,0.000000,0.0
9,19,0.000000,0.0
...,...,...,...
15632,26,0.000000,0.0
15661,32,0.000000,0.0
15701,17,0.000000,0.0
15716,30,0.366667,0.0


In [19]:
newDataFrame.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 177 entries, 0 to 15737
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   url_len         177 non-null    int64  
 1   url_ratioDG     177 non-null    float64
 2   domain_ratioDG  177 non-null    float64
dtypes: float64(2), int64(1)
memory usage: 5.5 KB


In [20]:
importanteIndex = ~newDataFrame.isin([np.nan, np.inf, -np.inf]).any(1)
newDataFrame[importanteIndex].astype(np.float32)

newDataFrame

,url_len,url_ratioDG,domain_ratioDG
0,20,0.000000,0.0
1,19,0.105263,0.0
2,42,0.023810,0.0
6,36,0.000000,0.0
9,19,0.000000,0.0
...,...,...,...
15632,26,0.000000,0.0
15661,32,0.000000,0.0
15701,17,0.000000,0.0
15716,30,0.366667,0.0


In [21]:
# Importancion del modelo clasificador
import joblib

model = joblib.load('modelo_lab1.h5')
rsults = model.predict(newDataFrame)

In [22]:
resultado = []
# se recorre cada resutlado de la prediccion y se agrega a la lista de resultado
for i in rsults:
    if i == 1:
        resultado.append(True)
    else:
        resultado.append(False)
        
print('Resultados de prediccion: ', resultado)

Resultados de prediccion:  [False, True, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, True, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, True, False, False, False, False, False, False, False, False, False, False, False, False, True, False, True, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, True, True, True, True, False, False, False, True, False, Fal

In [23]:
print("Dominiso DGA del modelo:")
dominios_unicos[resultado]['tld_extract_domain']

Dominiso DGA del modelo:


1                           dropbox.com
2                             aoltw.net
148                             aol.com
163            192.168.22.110phpmyadmin
554                           aoltw.net
661                         mozilla.com
856                          google.com
859                           aoltw.net
1106                        comcast.net
1195                   "192.168.206.56"
1701                        comcast.net
1836                    192.168.26-27.0
4496                         google.com
6762                         google.com
6799                     stayonline.net
6815                          aoltw.net
6898                    192.168.21.1201
7861                             me.com
9979                    1922.168.22.254
10048              1922.168.22.254.home
10607                   192.168.21-28.0
14730              192.168.21-28.0.home
15507                   192.168.22.201:
15716    192.168.22.201:.stayonline.net
Name: tld_extract_domain, dtype: object

In [24]:
### Clasificacion de dominios DGA por Flare
from flare.data_science.features import dga_classifier
dga_c = dga_classifier()

[*] Initializing... training classifier - Please wait.
[+] Training classifier on training set
[+] Out of sample legit f1 score 0.9988941431715974
[+] Out of sample dga f1 score 0.6307151230949589
[+] Training final classifier
[+] Classifier Ready


In [25]:
dominios_unicos['is_legit'] = dominios_unicos['tld_extract_domain'].apply(dga_c.predict)

In [26]:
dominios_unicos.head()

,timestamp,flow_id,pcap_cnt,event_type,vlan,src_ip,src_port,dest_ip,dest_port,proto,dns.type,dns.id,dns.rrname,dns.rrtype,dns.tx_id,dns.rcode,dns.ttl,dns.rdata,tld_extract_domain,is_legit
0,2017-07-22T17:33:16.661646-0500,1327836194150542,22269,dns,110,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,59680,2001:0500:0001:0000:0000:0000:803f:0235,53,UDP,query,15529,api.wunderground.com,A,0.0,NaN,NaN,NaN,wunderground.com,legit
1,2017-07-22T17:33:24.990320-0500,2022925111925872,54352,dns,110,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,38051,2001:0500:0003:0000:0000:0000:0000:0042,53,UDP,query,58278,stork79.dropbox.com,A,0.0,NaN,NaN,NaN,dropbox.com,legit
2,2017-07-22T17:33:27.379891-0500,578544790391795,54519,dns,150,192.168.205.170,31393,192.168.207.4,53,UDP,query,54724,hpca-tier2.office.aol.com.ad.aol.aoltw.net,A,0.0,NaN,NaN,NaN,aoltw.net,legit
6,2017-07-22T17:33:38.537426-0500,2167545251640146,55687,dns,180,192.168.198.62,35092,192.168.207.4,53,UDP,query,7425,safebrowsing.clients.google.com.home,A,0.0,NaN,NaN,NaN,safebrowsing.clients.google.com.home,legit
9,2017-07-22T17:33:45.672579-0500,541745511744323,56307,dns,120,192.168.202.172,55580,192.168.207.4,53,UDP,query,12854,fxfeeds.mozilla.com,A,0.0,NaN,NaN,NaN,mozilla.com,legit


In [27]:
dominios_unicos['is_legit'].value_counts()

legit    171
dga        6
Name: is_legit, dtype: int64

In [29]:
# Se filtran claisificados como DGA y muéstrelos.
dominios_unicos[dominios_unicos['is_legit'] == 'dga']['tld_extract_domain']

7963             msftncsi.com
8026             msftncsi.com
9856     vtlfccmfxlkgifuf.com
10048    1922.168.22.254.home
10646    ejfodfmfxlkgifuf.xyz
14730    192.168.21-28.0.home
Name: tld_extract_domain, dtype: object

**¿Son los mismos dominios que los de su modelo?**

No completamente, hay algunos que si como lo es 192.168.21-28.0.home , pero resto si son clasificados como DGA en uno en el otro no

#### Dominios en Umbrella Cisco

In [30]:
from flare.tools.umbrella import Umbrella
cisco_values = Umbrella(limit = 1000000)

[+] Updated Umbrella Top 1 Million list...


In [35]:
inCisco = []
notInCisco = []
for i in dominios_unicos[dominios_unicos['is_legit'] == 'dga']['tld_extract_domain']:
    umresult = cisco_values.domain_tld_in_umbrella(i)
    if umresult == True:
        inCisco.append(i)
    else:
        notInCisco.append(i)

print("Dominios en Cisco: ", inCisco)
print("Dominios que NO estan en Cisco: ", notInCisco)

Dominios en Cisco:  ['msftncsi.com', 'msftncsi.com']
Dominios que NO estan en Cisco:  ['vtlfccmfxlkgifuf.com', '1922.168.22.254.home', 'ejfodfmfxlkgifuf.xyz', '192.168.21-28.0.home']


#### Buscar fecha de creacion del dominio

In [36]:
import whois

def fecha_creacion_dom(dom):
    w = whois.whois(dom)
    return w.fecha_creacion_dom

In [44]:
withDate = []
notDate = []
for i in dominios_unicos[dominios_unicos['is_legit'] == 'dga']['tld_extract_domain']:
    try:
        whoistime = fecha_creacion_dom(i)
        if whoistime != None:
            withDate.append(i)
        else:
            notDate.append(i)
    except:
        print(i, "not found!")

print("Dominios con fecha de creacion: ", withDate)
print("Dominios que sin fecha de creacion: ", notDate)

vtlfccmfxlkgifuf.com not found!
Dominios con fecha de creacion:  []
Dominios que sin fecha de creacion:  ['msftncsi.com', 'msftncsi.com', '1922.168.22.254.home', 'ejfodfmfxlkgifuf.xyz', '192.168.21-28.0.home']


El dominio msftncsi.com y el 192.168.21-28.0.home carecen de tiempo de creacion. Ambos fueron clasificacados como dga pero unicamente msftncsi.com se encontro en los registros de Cisco Umbrella. Este entonces es un dominio potencialmente, mientras que 192.168.21-28.0.home es unicamente sospecho porque no aparece en los registros de Cisco Umbrella pero no tienen fecha de creacion.